# Convolutional SNN
### **Classifying Fashion-MNIST with Convolutional SNN**

This tutorial goes over how to train a convolutional spiking neural network (CSNN) on the Fashion-MNIST dataset and deploy on HiAER Spike using our conversion pipline.

### **Define a CSNN**
To build a CSNN with PyTorch, we can use snnTorch, SpikingJelly or other deep learning frameworks that are based on PyTorch. Currently, our conversion pipline supports snnTorch and SpikingJelly. In this tutorial, we will be using SpikingJelly.

Install the PyPi distribution of SpikingJelly

In [1]:
!pip install spikingjelly
!pip install torchvision torchviz torchview

# Library to interact with the CRI hardware
!pip install hs_api

Import necessary libraries from SpikingJelly and PyTorch

In [2]:
from spikingjelly.activation_based import neuron, functional, surrogate, layer
import torch 
import torch.nn as nn

### **Model Architecture**
Using SpikingJelly, we can define a CSNN with the architecture of 8C3-BN-6272FC10
- 8C3: a 3x3 convolutional kernel with 8 channels
- BN: batch normalization layer 
- 6272FC10: the fully connected output layer 
 
#### **Surrogate Function**
SpikingJelly and snnTorch both use backpropagation through time to train the spiking neural networks. However, because of the non-differentiability of spikes, surrogate gradients are used in place of the Heaviside function in the backward pass

In [3]:
class model(nn.Module): 
    def __init__(self, channels=8): 
        super().__init__()
        self.conv = nn.Conv2d(1, channels, kernel_size=3, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(channels)
        self.lif1 = neuron.IFNode(surrogate_function=surrogate.ATan())
        self.flat = nn.Flatten()
        self.linear = nn.Linear(channels * 28 * 28, 10, bias=False)
        self.lif2 = neuron.IFNode(surrogate_function=surrogate.ATan())
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.lif1(x)
        x = self.flat(x)
        x = self.linear(x)
        x = self.lif2(x)
        return x

In [4]:
#Initiate the Network
net = model()

from pprint import pprint
pprint(net)

#TODO: print out model architecture as a graph
# batch = next(iter(train_loader))
# yhat = model(batch) # Give dummy batch to forward().

# make_dot(yhat, params=dict(list(model.named_parameters()))).render("SCNN", format="png")


model(
  (conv): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lif1): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
    (surrogate_function): ATan(alpha=2.0, spiking=True)
  )
  (flat): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=6272, out_features=10, bias=False)
  (lif2): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
    (surrogate_function): ATan(alpha=2.0, spiking=True)
  )
)


### **Setting up the MNIST Dataset**

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Download Fashion-MNIST data from torch 
fashion_mnist_train = datasets.FashionMNIST('data/fashion_mnist', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))
fashion_mnist_test = datasets.FashionMNIST('data/fashion_mnist', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]))

# Create DataLoaders
train_loader = DataLoader(fashion_mnist_train, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=128, shuffle=True, drop_last=True)


### **Training the SNN**
Since we are using a static image dataset, we will first encode the image into spikes using the rate encoding function from spikingjelly. With rate encoding, the input feature determines the firing frequency and the neuron that fries the most is selected as the predicted class.  

In [6]:
from spikingjelly.activation_based import encoding
import time

In [7]:
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())


True
True


In [8]:
#Setting up the encoder and the time steps
encoder = encoding.PoissonEncoder()
num_steps = 40

#Define training parameters
epochs = 20
dtype = torch.float
device = torch.device("mps") # if torch.cuda.is_available() else torch.device("cpu")
print(device)

#Copy netowrk to device 
net.to(device)

#Define optimizer, scheduler and the loss function
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
loss_fun = torch.nn.MSELoss()

mps


In [9]:
for epoch in range(epochs):
    start_time = time.time()
    net.train()
    train_loss = 0
    train_acc = 0
    train_samples = 0
    for img, label in train_loader:
        optimizer.zero_grad()
        img = img.to(device)
        label = label.to(device)
        label_onehot = torch.nn.functional.one_hot(label, 10).float()
        out_fr = 0.
        for t in range(num_steps):
            encoded_img = encoder(img)
            out_fr += net(encoded_img)
        out_fr = out_fr/num_steps  
        loss = loss_fun(out_fr, label_onehot)
        loss.backward()
        optimizer.step()

        train_samples += label.numel()
        train_loss += loss.item() * label.numel()
        train_acc += (out_fr.argmax(1) == label).float().sum().item()

        #reset the membrane protential after each input image
        functional.reset_net(net)

    train_time = time.time()
    train_speed = train_samples / (train_time - start_time)
    train_loss /= train_samples
    train_acc /= train_samples
    
    lr_scheduler.step()
        
    net.eval()
    test_loss = 0
    test_acc = 0
    test_samples = 0

    with torch.no_grad():
        for img, label in test_loader:
            img = img.to(device)
            label = label.to(device)
            label_onehot = torch.nn.functional.one_hot(label, 10).float()
            out_fr = 0.   
            for t in range(num_steps):
                encoded_img = encoder(img)
                out_fr += net(encoded_img)
            out_fr = out_fr/num_steps  

            loss = loss_fun(out_fr, label_onehot)

            test_samples += label.numel()
            test_loss += loss.item() * label.numel()
            test_acc += (out_fr.argmax(1) == label).float().sum().item()
            functional.reset_net(net)

    test_time = time.time()
    test_speed = test_samples / (test_time - train_time)
    test_loss /= test_samples
    test_acc /= test_samples
    
    
    print(f'epoch = {epoch}, train_loss ={train_loss: .4f}, train_acc ={train_acc: .4f}, test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}')
    print(f'train speed ={train_speed: .4f} images/s, test speed ={test_speed: .4f} images/s')

epoch = 0, train_loss = 0.0271, train_acc = 0.8288, test_loss = 0.0232, test_acc = 0.8513
train speed = 822.3998 images/s, test speed = 2005.7721 images/s
epoch = 1, train_loss = 0.0206, train_acc = 0.8728, test_loss = 0.0212, test_acc = 0.8661
train speed = 838.0144 images/s, test speed = 2085.9205 images/s
epoch = 2, train_loss = 0.0189, train_acc = 0.8844, test_loss = 0.0204, test_acc = 0.8715
train speed = 851.7008 images/s, test speed = 2122.0700 images/s
epoch = 3, train_loss = 0.0178, train_acc = 0.8902, test_loss = 0.0196, test_acc = 0.8753
train speed = 837.6941 images/s, test speed = 2034.1627 images/s
epoch = 4, train_loss = 0.0171, train_acc = 0.8951, test_loss = 0.0192, test_acc = 0.8768
train speed = 824.7459 images/s, test speed = 2091.6778 images/s
epoch = 5, train_loss = 0.0165, train_acc = 0.8989, test_loss = 0.0192, test_acc = 0.8780
train speed = 827.2133 images/s, test speed = 2051.0987 images/s
epoch = 6, train_loss = 0.0159, train_acc = 0.9043, test_loss = 0.0191

In [ ]:
torch.save(net, 'model.pt')

### **Converting the trained SNN to HiAER Spike Format**

In [ ]:
from hs_api.converter import CRI_Converter, Quantize_Network, BN_Folder
from hs_api.api import CRI_network
# import hs_bridge #Uncomment when running on FPGA

#Fold the BN layer 
bn = BN_Folder() 
net_bn = bn.fold(net)

#Weight, Bias Quantization 
qn = Quantize_Network() 
net_quan = qn.quantize(net_bn)

#Set the parameters for conversion
input_layer = 0 #first pytorch layer that acts as synapses
output_layer = 4 #last pytorch layer that acts as synapses
input_shape = (1, 28, 28)
backend = 'spikingjelly'
v_threshold = qn.v_threshold

cn = CRI_Converter(num_steps = num_steps, 
                   input_layer = input_layer, 
                   output_layer = output_layer, 
                   input_shape = input_shape,
                   backend=backend,
                   v_threshold = v_threshold)
cn.layer_converter(net_quan)

### **Initiate the HiAER-Spike SNN**

In [ ]:
config = {}
config['neuron_type'] = "I&F"
config['global_neuron_params'] = {}
config['global_neuron_params']['v_thr'] = int(quan_fun.v_threshold)
    
#Create a network running on the FPGA
hardwareNetwork = CRI_network(dict(cri_convert.axon_dict),
                              connections=dict(cri_convert.neuron_dict),
                              config=config,
                              target='CRI', 
                              outputs = cri_convert.output_neurons,
                              coreID=1)

#Create a network running on the software simulation
softwareNetwork = CRI_network(dict(cri_convert.axon_dict),
                              connections=dict(cri_convert.neuron_dict),
                              config=config,
                              target='simpleSim', 
                              outputs = cri_convert.output_neurons,
                              coreID=1)

### **Deploying the SNN on HiAER Spike**

Using the run_CRI_hw and run_CRI_sw method from the CRI_Converter class, we can deploy the converted SNN on the HiAER Spike platform.

In [ ]:
cri_convert.bias_start_idx = int(cri_convert.output_neurons[0])
loss_fun = nn.MSELoss()
start_time = time.time()
test_loss = 0
test_acc = 0
test_samples = 0
num_batches = 0

RUN_HARDWARE = False #Set to True if running on FPGA

for img, label in tqdm(test_loader):
    cri_input = cri_convert.input_converter(img)
    output = None
    if RUN_HARDWARE:
        output = torch.tensor(cri_convert.run_CRI_hw(cri_input,hardwareNetwork), dtype=float)
    else:
        output = torch.tensor(cri_convert.run_CRI_sw(cri_input,softwareNetwork), dtype=float)
    loss = loss_fun(output, label)
    test_samples += label.numel()
    test_loss += loss.item() * label.numel()
    test_acc += (output == label).float().sum().item()
    num_batches += 1
test_time = time.time()
test_speed = test_samples / (test_time - start_time)
test_loss /= test_samples
test_acc /= test_samples

print(f'test_loss ={test_loss: .4f}, test_acc ={test_acc: .4f}')
print(f'test speed ={test_speed: .4f} images/s')